In [4]:
import pandas as pd
from sklearn.preprocessing import minmax_scale
import numpy as np
from scipy.stats import multivariate_normal
import sklearn as sk
from sklearn import covariance
from sklearn.metrics import confusion_matrix, classification_report

def Kernel(x_0,x,h):
    n = len(x_0)
    dist = np.zeros(n)
    for i in range(n):
        dist[i]=abs(x_0[i]-x[i])/h
    if all(i <= 0.5 for i in dist):
        return 1
    else:
        return 0

def parzenprob(x_0,cl,h):
    n = len(x_0)
    V = h**n
    num = len(cl)
    denum = V*num
    p = 0
    for i in range(num):
        p = p+Kernel(x_0,cl[i][:],h)
    
    return p/denum
    
trainData = pd.read_csv('pendigitstrain.csv')
testData = pd.read_csv('pendigitstest.csv')

TrainLabel = trainData['17']
TestLabel = testData['17']

TrainFeat = trainData.drop(['17'],axis = 1)
TestFeat = testData.drop(['17'],axis = 1)

TrainFeatScaled = minmax_scale(TrainFeat, feature_range = (0,1))
TestFeatScaled = minmax_scale(TestFeat, feature_range = (0,1))


def decision(x, X, Y):
    probability_List = []
    num_classes = len(np.unique(Y))
    shrinkage = 0.15
    N_T = len(X)
    h = 1
    for i in range(num_classes):
        X_train = X[Y==i]
        probability = parzenprob(x,X_train,h)*(len(X_train)/N_T)
        probability_List.append(probability)
    return probability_List.index(max(probability_List))



TestDecision = np.zeros((len(TestLabel),1))
CCR = 0
for i in range(len(TestDecision)):
    x = TestFeatScaled[i][:]
    TestDecision[i][0]=decision(x, TrainFeatScaled, TrainLabel)
    if TestDecision[i][0] == TestLabel[i]:
        CCR += 1



print(CCR/len(TestLabel))

print(confusion_matrix(TestDecision, TestLabel, [0, 1,2,3,4,5,6,7,8,9]))
print(classification_report(TestDecision, TestLabel))


0.7715837621497998
[[353   0   0   0   0   0   0   0  15   0]
 [  0 104   4   7   0   0   0  52   0   9]
 [  2 122 339   0   2   0   0 114   0   0]
 [  0  46   0 327   0  63   1  24   0  56]
 [  0   1   0   0 329   0   0   3   1  15]
 [  0   3   0   0   1 207   1   3  24   9]
 [  1  49   0   0  28  30 333   0   1   0]
 [  0  36  21   0   3   0   0 166   0   0]
 [  7   0   0   0   0   0   1   2 295   1]
 [  0   3   0   2   1  35   0   0   0 246]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       368
         1.0       0.29      0.59      0.39       176
         2.0       0.93      0.59      0.72       579
         3.0       0.97      0.63      0.77       517
         4.0       0.90      0.94      0.92       349
         5.0       0.62      0.83      0.71       248
         6.0       0.99      0.75      0.86       442
         7.0       0.46      0.73      0.56       226
         8.0       0.88      0.96      0.92       306
         

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [5]:
from sklearn.neighbors import RadiusNeighborsClassifier

RNC = RadiusNeighborsClassifier(radius=1, metric='minkowski', p=2)

RNC.fit(TrainFeatScaled, TrainLabel)

TestDecision = RNC.predict(TestFeatScaled)

print(confusion_matrix(TestDecision, TestLabel, [0, 1,2,3,4,5,6,7,8,9]))
print(classification_report(TestDecision, TestLabel))

[[342   0   0   0   0   0   0   0   7   0]
 [  0 132   3   3   0   0   0  53   0   9]
 [  0 153 358   0   0   0   0 113   0   0]
 [  0  20   0 332   0  98   0  12   0  94]
 [  0   1   0   0 336   0   0   2   0  13]
 [  0  37   0   0   2 181   0   2  14   5]
 [ 18  17   0   0  25  24 336   1   0   0]
 [  0   4   3   0   0   0   0 181   2   0]
 [  2   0   0   0   0   0   0   0 313   1]
 [  1   0   0   1   1  32   0   0   0 214]]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       349
           1       0.36      0.66      0.47       200
           2       0.98      0.57      0.72       624
           3       0.99      0.60      0.74       556
           4       0.92      0.95      0.94       352
           5       0.54      0.75      0.63       241
           6       1.00      0.80      0.89       421
           7       0.50      0.95      0.65       190
           8       0.93      0.99      0.96       316
           9       0.64     

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
